# Retrieval

The steps are:
1. Load documents
2. Split documents
3. Embed & store documents
4. Retrieve from documents

## 0. Prerequisites

In [18]:
!pip install langchain openai GitPython chromadb unstructured markdown tiktoken langchainhub


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [19]:
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
search_keyword = "AlwaysBehavior"

## 1. Load documents

In [21]:
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import UnstructuredMarkdownLoader

docs_loader = DirectoryLoader("pronto-docs", glob="**/*.md", loader_cls=UnstructuredMarkdownLoader)
docs = docs_loader.load()

## 2. Split documents

In [22]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

docs_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=100, chunk_overlap=0)

In [23]:
docs_splits = docs_splitter.split_documents(docs)

In [24]:
len([d for d in docs_splits if d.metadata["source"] == "pronto-docs/README.md"])

285

In [25]:
len(docs_splits)

912

In [26]:
from langchain_core.documents import Document
from langchain.text_splitter import TextSplitter

def get_splits_from_keyword(keyword: str, splitter: TextSplitter, documents: list[Document]):
    return list(filter(lambda x: keyword in x.page_content, splitter.split_documents(documents)))

In [27]:
chunk_size = 100
chunk_overlap = 0

### 2.1 Recursive Character Text Splitter

In [28]:
rct_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
)

In [29]:
get_splits_from_keyword("ValueBehavior", rct_splitter, docs)

[Document(page_content='ValueBehavior', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='[code]false[/code], only children ValueBehaviors are added to the menu (if they are visible)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content='create_ui_for_value\n\ngdscript\nfunc create_ui_for_value(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content='create_ui_for_value_enum\n\ngdscript\nfunc create_ui_for_value_enum(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content='create_ui_for_value_bool\n\ngdscript\nfunc create_ui_for_value_bool(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content='gdscript\nfunc create_ui_slider_for_value_float(value: ValueBehavior)\n\ncreate_minimizing_button', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content

### 2.2 Recursive Character Text Splitter (from language Markdown)

In [30]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language

rctm_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=chunk_size, chunk_overlap=chunk_overlap)

In [31]:
get_splits_from_keyword("ValueBehavior", rctm_splitter, docs)

[Document(page_content='ValueBehavior', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='[code]false[/code], only children ValueBehaviors are added to the menu (if they are visible)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content='create_ui_for_value\n\ngdscript\nfunc create_ui_for_value(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content='create_ui_for_value_enum\n\ngdscript\nfunc create_ui_for_value_enum(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content='create_ui_for_value_bool\n\ngdscript\nfunc create_ui_for_value_bool(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content='gdscript\nfunc create_ui_slider_for_value_float(value: ValueBehavior)\n\ncreate_minimizing_button', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}),
 Document(page_content

### 2.3 Recursive Character Text Splitter (custom separators)

In [32]:
rctcs_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    separators=["#", "##", "###", "\n\n", "\n \n", "\n"]
)

In [33]:
rctcs_splitter.split_documents(filter(lambda x: x.metadata["source"] == "pronto-docs/README.md", docs))

[Document(page_content='Pronto', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='\nPronto is a framework for Godot to make prototyping game mechanics faster. It is not a framework aimed at helping to create entire games faster. The resulting prototypes are a means to quickly explore ideas, throw away the prototype, and only properly implement ideas that turned out well.', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='Play prototypes created in pronto here.\n\nFunction', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='\nThe main idea of Pronto is to make behavior visible. Our hypothesis is that it will be easier to create and tweak game mechanics. For example, instead of defining numbers in code for the distance a platform moves, we use handles in the game world to visually direct it.', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='\nPronto consists of a set of Godot Nodes called Behavior tha

In [34]:
splits = get_splits_from_keyword("ValueBehavior", rctcs_splitter, docs)
print(len(splits))
print(splits)

10
[Document(page_content='ValueBehavior', metadata={'source': 'pronto-docs/README.md'}), Document(page_content='\nIf [code]true[/code], all values in the scene are added to the menu (if they are visible) [br]If [code]false[/code], only children ValueBehaviors are added to the menu (if they are visible)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}), Document(page_content='create_ui_for_value\n\ngdscript\nfunc create_ui_for_value(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}), Document(page_content='create_ui_for_value_enum\n\ngdscript\nfunc create_ui_for_value_enum(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}), Document(page_content='create_ui_for_value_bool\n\ngdscript\nfunc create_ui_for_value_bool(value: ValueBehavior)', metadata={'source': 'pronto-docs/PrototypingUIBehavior.md'}), Document(page_content='gdscript\nfunc create_ui_slider_for_value_float(value: ValueBehavior)\n\ncreate_minimiz

In [35]:
from langchain.text_splitter import CharacterTextSplitter

ct_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator="\n"
)

In [36]:
get_splits_from_keyword("connect", ct_splitter, docs)

Created a chunk of size 120, which is longer than the specified 100
Created a chunk of size 131, which is longer than the specified 100
Created a chunk of size 105, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 113, which is longer than the specified 100
Created a chunk of size 154, which is longer than the specified 100
Created a chunk of size 138, which is longer than the specified 100
Created a chunk of size 146, which is longer than the specified 100
Created a chunk of size 125, which is longer than the specified 100
Created a chunk of size 125, which is longer than the specified 100
Created a chunk of size 205, which is longer than the specified 100
Created a chunk of size 123, which is longer than the specified 100
Created a chunk of size 114, which is longer than the specified 100
Created a chunk of size 168, which is longer than the specified 100
Created a chunk of size 106, which is longer tha

[Document(page_content='line_text_function\ngdscript\nfunc line_text_function(connection: Connection) -> Callable', metadata={'source': 'pronto-docs/StateBehavior.md'}),
 Document(page_content='Allows you to define a template subtree of Nodes that you want to repeat multiple times without copy-pasting. Add your template as a child of the Instance node, then hover the connection dialog and click the "Instance" button. Note: internally, this creates a "hidden" scene that you need to commit as well. You can thus use  "Editable children"  in Godot by right-clicking the instance and tweaking properties while inheriting the rest.', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='VisualLineBehavior \n Show a colored Line between two Nodes. Useful as a quick visual connection.', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='You can use the set function to modify any properties when a connection triggers.\nJuice', metadata={'source': 'pronto-docs/

## 3. Embed & store documents

This is where it gets interesting. We have different options for retrieving:
- Vectorstore
- Parent Document
- Multi-Vector
- Self Query
- Contextual Compression
- Time-Weighted Vectorstore
- Multi-Query Retriever
- Ensemble
- Long-Context Reorder

In [37]:
from langchain_community.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [38]:
from langchain_community.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [39]:
from langchain_community.vectorstores.chroma import Chroma

db = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory="./chroma_db")

In [40]:
from langchain_core.retrievers import BaseRetriever

def check_for_keyword(keyword: str, retriever: BaseRetriever):
    rel_docs = retriever.get_relevant_documents(keyword)
    count_results = sum(map(lambda x: int(keyword in x.page_content), rel_docs))
    return str(count_results) + "/" + str(len(rel_docs))

### 3.1 Vectorstore

This is the basic method directly build into the vectorstore.

In [41]:
vec_retriever = db.as_retriever()
vec_retriever_mmr = db.as_retriever(
    search_type="mmr",  # Also test "similarity",
)

In [42]:
vec_retriever.get_relevant_documents(search_keyword)

[Document(page_content='StateMachineBehavior\n\nExtends: Behavior\n\nDescription', metadata={'source': 'pronto-docs/StateMachineBehavior.md'}),
 Document(page_content='StateMachineBehavior\n\nExtends: Behavior\n\nDescription', metadata={'source': 'pronto-docs/StateMachineBehavior.md'}),
 Document(page_content='StateMachineBehavior\n\nExtends: Behavior\n\nDescription', metadata={'source': 'pronto-docs/StateMachineBehavior.md'}),
 Document(page_content='StateMachineBehavior\n\nExtends: Behavior\n\nDescription', metadata={'source': 'pronto-docs/StateMachineBehavior.md'})]

In [43]:
print(check_for_keyword(search_keyword, vec_retriever))
print(check_for_keyword(search_keyword, vec_retriever_mmr))

0/4
1/4


As you can see the vectorstore method delivers some correct result but performs poorly.

### 3.2 Parent Document Retriever

#### 3.2.1 Full Documents from smaller chunks

In [44]:
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

pdfd_vectorstore = Chroma(
    collection_name="full_documents", embedding_function=OpenAIEmbeddings()
)

pdfd_store = InMemoryStore()
pdfd_retriever = ParentDocumentRetriever(
    vectorstore=pdfd_vectorstore,
    docstore=pdfd_store,
    child_splitter=docs_splitter,
)

In [45]:
pdfd_retriever.add_documents(docs, ids=None)

In [46]:
pdfd_retriever.get_relevant_documents(search_keyword)

[Document(page_content='AlwaysBehavior\n\nExtends: Behavior\n\nDescription\n\nSee [signal AlwaysBehavior.always] and [signal AlwaysBehavior.physics_always] for a description of which signal to listen to.\n\nSee [signal AlwaysBehavior.always] and [signal AlwaysBehavior.physics_always] for a description of which signal to listen to.\n\nProperty Descriptions\n\npaused\n\ngdscript\n@export var paused = false\n\nIf this is set to [code]true[/code] the AlwaysBehavior will stop emitting any signals. During runtime, do not set this directly. Instead use [method AlwaysBehavior.pause] and [method AlwaysBehavior.resume]\n\nMethod Descriptions\n\npause\n\ngdscript\nfunc pause()\n\nCalling this method results in pausing the execution of always Use [method AlwaysBehavior.resume] to continue the execution\n\nresume\n\ngdscript\nfunc resume()\n\nCalling this method results in continuing the paused execution Use [method AlwaysBehavior.pause] to stop it again.\n\nSignals\n\nsignal always(delta): This Si

In [47]:
print(check_for_keyword(search_keyword, pdfd_retriever))

2/2


This method outright delivers a too large chunk size i.e. the whole document and is therefore not suitable.

#### 3.2.2 Larger chunks from smaller chunks

In [48]:
from langchain.retrievers.multi_vector import SearchType
from langchain.text_splitter import RecursiveCharacterTextSplitter, Language
from langchain_community.vectorstores.chroma import Chroma
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langchain_community.embeddings import OpenAIEmbeddings

parent_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=400, chunk_overlap=0)
child_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.MARKDOWN, chunk_size=100, chunk_overlap=0)

pdlg_vectorstore = Chroma(
    collection_name="split_parents", embedding_function=OpenAIEmbeddings()
)
pdlg_store = InMemoryStore()
pdlg_retriever = ParentDocumentRetriever(
    vectorstore=pdlg_vectorstore,
    docstore=pdlg_store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

pdlg_retriever.add_documents(docs)

In [49]:
pdlg_retriever.get_relevant_documents("How to access values in a StoreBehaviour?")

[Document(page_content="The StateBehavior is the fundamental building block of a state machine. Each StateBehavior emits the signals  StateBehavior.entered()  and  StateBehavior.exited()  to communicate the state machine's state \n \n StoreBehavior \n Use the Godot meta properties to store state. You can configure it to store values in the global dictionary  G  and access it via  G.at(prop) . \n \n ValueBehavior", metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content="In ExpressionInspector.gd you can specify that a property should get a GDScript code editor.\n\nHints on Designing new Behaviors\n\nDon't bundle too much: give each concern its own behavior, e.g. appearance, movement, shooting. Design behaviors such that they are easy to connect to one another instead of pre-packaging them with lots of concerns.\n\nPhase 3: Deploying your Prototype/Game", metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='When triggered, moves its parent. Can be set to 

### 3.3 MultiQueryRetriever

In [50]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

#### 3.3.1 Vectorstore as Base Retriever

In [51]:
from langchain.retrievers import MultiQueryRetriever

v_mq_retriever = MultiQueryRetriever.from_llm(
    retriever=db.as_retriever(), llm=llm
)

In [52]:
v_mq_retriever.get_relevant_documents(query="How to change the sprite of a PlaceholderBehavior?")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the steps to modify the sprite of a PlaceholderBehavior?', '2. Can you explain the process of replacing the sprite in a PlaceholderBehavior?', '3. Are there any specific instructions for updating the sprite in a PlaceholderBehavior?']


[Document(page_content="PlaceholderBehavior \n Show a colored rectangle with a label. Useful as a quick means to communicate a game object's function. Functions as a collision shape, so you don't need to add another. Instead of a rectangle a placeholder can also display a sprite instead (use the Sprite Library in the Inspector to choose an existing texture or load your own). Can be  flash() ed in a different color.", metadata={'source': 'pronto/README.md'}),
 Document(page_content="PlaceholderBehavior \n Show a colored rectangle with a label. Useful as a quick means to communicate a game object's function. Functions as a collision shape, so you don't need to add another. Instead of a rectangle a placeholder can also display a sprite instead (use the Sprite Library in the Inspector to choose an existing texture or load your own). Can be  flash() ed in a different color.", metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='PlaceholderBehavior\n\nExtends: Behavior\n\nD

#### 3.3.2 Parent Document Retriever (Larger Chunks) as Base Retriever

In [53]:
from langchain.retrievers import MultiQueryRetriever

pdlg_mq_retriever = MultiQueryRetriever.from_llm(
    retriever=pdlg_retriever, llm=llm
)

In [54]:
pdlg_mq_output_docs = pdlg_mq_retriever.get_relevant_documents(query="How to delete a connection of two Behaviors?")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the process for removing a connection between two Behaviors?', '2. Can you explain the steps involved in deleting the link between two Behaviors?', '3. Is there a specific method or command to disconnect two Behaviors?']


In [55]:
check_for_keyword("PlaceholderBehavior", pdlg_mq_retriever)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How does the placeholder behavior work?', '2. Can you explain the behavior of placeholders?', '3. What is the purpose of using placeholders and how do they behave?']


'1/5'

### 3.4 Long-Context Reorder

In [56]:
from langchain_community.document_transformers import LongContextReorder

reordering = LongContextReorder()
reordered_docs = reordering.transform_documents(pdlg_mq_output_docs)

In [57]:
reordered_docs

[Document(page_content='Connections are an extension of Godot signals to be more flexible. Connections can be dragged from any behavior to any arbitrary node in a scene. They are the primary means to assemble your game by wiring Behaviors together.\n\nCreating\n\nConnections are created by hovering the "+" that appears below selected nodes. There are two types of connections: Target Connections and Expression Connections.', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='You can disable a connection by unchecking the checkbox in the top-right or in its context menu.\n\nYou can also pin a connection editor so that it remains in the scene after saving them. You can click and drag editors in the scene as well. You can drag the handles at the bottom right of each text field to resize the editor.', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='30-90 seconds (no need to cut the video, can be shaky and raw)\n\nshow what you did and what insights

### 3.5 Ensemble Retriever

In [58]:
from langchain.retrievers import EnsembleRetriever

ensemble_retriever = EnsembleRetriever(
    retrievers=[pdlg_mq_retriever, pdlg_retriever], weights=[0.5, 0.5]
)

In [59]:
reordered_docs = reordering.transform_documents(ensemble_retriever.get_relevant_documents("AlwaysBehavior"))

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the typical behaviors that are always exhibited?', '2. Can you provide examples of behaviors that are consistently observed?', '3. Are there any specific actions or conduct that are consistently followed?']


In [60]:
reordered_docs

[Document(page_content='CollisionBehavior \n Triggers when something collides with its parent. The  collided  signal also provides the  direction  of the collision pointing in the direction from the parent to the node it collided with. NOTE: only works for Area2D, RigidBody2D and CharacterBody2D as of now. \n \n ControlsBehavior', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='AlwaysBehavior\n\nExtends: Behavior\n\nDescription\n\nSee [signal AlwaysBehavior.always] and [signal AlwaysBehavior.physics_always] for a description of which signal to listen to.\n\nSee [signal AlwaysBehavior.always] and [signal AlwaysBehavior.physics_always] for a description of which signal to listen to.\n\nProperty Descriptions\n\npaused\n\ngdscript\n@export var paused = false', metadata={'source': 'pronto-docs/AlwaysBehavior.md'}),
 Document(page_content='The following list of behaviors primarily cause effects when triggered.', metadata={'source': 'pronto-docs/README.md'}),
 Document(

### 3.6 Contextual Compression

In [61]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=ensemble_retriever
)

In [62]:
compression_retriever.get_relevant_documents("AlwaysBehavior")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the typical behaviors that are always exhibited?', '2. Can you provide examples of behaviors that are consistently observed?', '3. Are there any specific actions or conduct that are consistently followed?']
/Users/benjamindaniel/Developer/PycharmProjects/PromptoExperimentation/venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/benjamindaniel/Developer/PycharmProjects/PromptoExperimentation/venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/benjamindaniel/Developer/PycharmProjects/PromptoExperimentation/venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass a

[Document(page_content="You can return a list of handles (knobs on the canvas that you can move). See Placeholder and Value behaviors for examples.\n\nHandles can exist in local space of the nodes in the canvas or in the space of the overlay (independent of the canvas' zoom).\n\nLines and Text Below a Behavior", metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='AlwaysBehavior\n\nExtends: Behavior\n\nDescription\n\nSee [signal AlwaysBehavior.always] and [signal AlwaysBehavior.physics_always] for a description of which signal to listen to.\n\nSee [signal AlwaysBehavior.always] and [signal AlwaysBehavior.physics_always] for a description of which signal to listen to.\n\nProperty Descriptions\n\npaused\n\ngdscript\n@export var paused = false', metadata={'source': 'pronto-docs/AlwaysBehavior.md'}),
 Document(page_content='The following list of behaviors primarily cause effects when triggered.', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='The f

In [63]:
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
contextual_compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=ensemble_retriever
)

In [64]:
contextual_compression_retriever.get_relevant_documents("AlwaysBehavior")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What are the typical behaviors that are always exhibited?', '2. Can you provide examples of behaviors that are consistently observed?', '3. Are there any specific actions or conduct that are consistently followed?']
/Users/benjamindaniel/Developer/PycharmProjects/PromptoExperimentation/venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/benjamindaniel/Developer/PycharmProjects/PromptoExperimentation/venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(
/Users/benjamindaniel/Developer/PycharmProjects/PromptoExperimentation/venv/lib/python3.9/site-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass a

[Document(page_content='CollisionBehavior \n Triggers when something collides with its parent. The  collided  signal also provides the  direction  of the collision pointing in the direction from the parent to the node it collided with.', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content="You can return a list of handles (knobs on the canvas that you can move). See Placeholder and Value behaviors for examples.\n\nHandles can exist in local space of the nodes in the canvas or in the space of the overlay (independent of the canvas' zoom).", metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='AlwaysBehavior\nExtends: Behavior\n\nDescription\n\nSee [signal AlwaysBehavior.always] and [signal AlwaysBehavior.physics_always] for a description of which signal to listen to.\n\nSee [signal AlwaysBehavior.always] and [signal AlwaysBehavior.physics_always] for a description of which signal to listen to.\n\nProperty Descriptions\n\npaused\n\ngdscript\n@export va

## 4. Agent

In [65]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    pdlg_retriever,
    "Intermediate Answer",
    "useful for when you need to ask with search"
)
tools = [tool]

In [66]:
from langchain.agents import initialize_agent, AgentType

agent = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True, handle_parsing_errors=True)

In [67]:
agent.run("Pronto is a prototyping framework for the Godot Game Engine. The following question relates to this framework. When answering, please search for anything related to either the framework or behaviors that are used to create gameplay mechanics in the framework. Now the question is as follows: How to change the sprite of a PlaceholderBehavior?")



> Entering new AgentExecutor chain...
Could not parse output: Yes.
Intermediate answer: Invalid or incomplete response
Could not parse output: I'm sorry, but I couldn't find any information on how to change the sprite of a PlaceholderBehavior in the Pronto prototyping framework for the Godot Game Engine.
Intermediate answer: Invalid or incomplete response
So the final answer is: I couldn't find any information on how to change the sprite of a PlaceholderBehavior in the Pronto prototyping framework for the Godot Game Engine.

> Finished chain.


"I couldn't find any information on how to change the sprite of a PlaceholderBehavior in the Pronto prototyping framework for the Godot Game Engine."

In [85]:
ensemble_retriever.get_relevant_documents("damage health")

INFO:langchain.retrievers.multi_query:Generated queries: ['1. How can I improve my health after experiencing damage?', '2. What are some ways to restore and maintain good health after sustaining damage?', '3. Are there any specific remedies or treatments to address health issues caused by damage?']


[Document(page_content='30-90 seconds (no need to cut the video, can be shaky and raw)\n\nshow what you did and what insights you gained\n\nyou are encouraged to open the game with a couple different parameter configurations and show the impact\n\nPhase 2: Extending the Framework\n\nWriting a New Behavior', metadata={'source': 'pronto-docs/README.md'}),
 Document(page_content='Setter: @invulnerability_color_setter\n\nsize\n\ngdscript\nvar size: Vector2\n\nGetter: @size_getter\n\nThe height and width of the health bar\n\nMethod Descriptions\n\ndamage\n\ngdscript\nfunc damage(amount)\n\nReduces the current health value by the given [code]amount[/code].\n\nheal\n\ngdscript\nfunc heal(amount)\n\nIncreases the current health value by the given [code]amount[/code].\n\nheal_full', metadata={'source': 'pronto-docs/HealthBarBehavior.md'}),
 Document(page_content='Show a rectangular health bar. Offers methods to manipulate the current health and emits a signal when health drops to zero. \n \n In

In [82]:
from langchain.memory import ConversationSummaryMemory

memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)

In [79]:
from langchain_core.prompts import PromptTemplate

prompt_template = """You are a helpful assistant answering all questions related to Pronto, a game mechanic prototyping framework for the Godot Game Engine. This framework introduces the concept of Behaviors. Behaviors drive the actions of game objects in a game. They can be connected to each other to create more complex interactions. Try to answer the question using the Pronto framework before thinking of using the Godot Engine. Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:"""
QA_PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [83]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=ensemble_retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": QA_PROMPT}
)

In [84]:
print(qa("How can I damage a player?"))

INFO:langchain.retrievers.multi_query:Generated queries: ["1. What are some strategies for negatively impacting a player's performance?", "2. Are there any tactics or techniques to hinder a player's progress or success?", "3. In what ways can I disrupt or impede a player's gameplay experience?"]


{'question': 'How can I damage a player?', 'chat_history': [SystemMessage(content='')], 'answer': "To harm a player using the Pronto framework, you can create a new behavior that triggers the harm effect when certain conditions are met. \n\nFirst, you would need to create a new behavior file and extend it from the Behavior class. Inside this behavior, you can define properties and signals that will be used to harm the player.\n\nNext, you can use the SceneRootBehavior to search for the player node in the scene. Once you have found the player node, you can emit a signal to harm the player. This signal can be connected to other behaviors or scripts to apply the harm effect, such as reducing the player's health or applying a negative status effect.\n\nYou can also use the Value behavior to expose the crucial variation points of the harm effect, such as the amount of damage or the type of harm. This allows you to easily adjust these parameters in the editor without modifying the behavior c